In [1]:
from utils import read_images
import pandas as pd
import numpy as np
import tifffile
import pyclesperanto_prototype as cle  # version 0.24.1
import napari_segment_blobs_and_things_with_membranes as nsbatwm  # version 0.3.6
from skimage import measure
from skimage.color import rgb2gray
from apoc import ObjectSegmenter, ObjectClassifier
from scipy import ndimage as ndi
import napari

In [2]:
# Define the directory containing your files
directory_path ="../data/test_data"

# The following function will read all the images contained within the directory_path above
# and store them grouped by well_id.
images_per_well = read_images(directory_path)

In [ ]:
# This is just a check, it prints the grouped z-stacks
for well_id, files in images_per_well.items():
    print(f'Well {well_id}:')
    for file_path in files:
        print(file_path)

In [ ]:
imgs_to_process = images_per_well["A01"]

all_in_focus_labels = []  # Store in-focus labels for merging

for input_img in imgs_to_process:

    # Load one RGB image and transform it into grayscale (if needed) for APOC
    rgb_img = tifffile.imread(input_img, is_ome=False)
    if len(rgb_img.shape) < 3:
        img = rgb_img
    elif rgb_img.shape[2] == 3:
        img = rgb2gray(rgb_img)
    else:
        print(
            "Modify the loader to accommodate different file formats",
            rgb_img.shape,
        )

    # Apply object segmenter from APOC
    try:
        segmenter = ObjectSegmenter(opencl_filename="../ObjectSegmenter.cl")
        result = segmenter.predict(image=img)
    except IndexError:
        segmenter = ObjectSegmenter(
            opencl_filename="../pretrained_APOC/ObjectSegmenter.cl"
        )
        result = segmenter.predict(image=img)

    # Closing some holes in the organoid labels
    closed_labels = cle.closing_labels(result, None, radius=4.0)

    # Exclude small labels, cutout in pixel area seems to be below 1000px
    exclude_small = cle.exclude_small_labels(closed_labels, None, 1000.0)
    exclude_small = np.array(
        exclude_small, dtype=np.int32
    )  # Change dtype of closed labels to feed array into nsbatwm.split

    # Splitting organoids into a binary mask
    split_organoids = nsbatwm.split_touching_objects(exclude_small, sigma=10.0)

    # Connected component (cc) labeling
    cc_split_organoids = nsbatwm.connected_component_labeling(
        split_organoids, False
    )

    # Apply object classifier from APOC
    try:
        classifier = ObjectClassifier(opencl_filename="../ObjectClassifier.cl")
        result = classifier.predict(labels=cc_split_organoids, image=img)
    except AttributeError:
        classifier = ObjectClassifier(
            opencl_filename="../pretrained_APOC/ObjectClassifier.cl"
        )
        result = classifier.predict(labels=cc_split_organoids, image=img)

    # Convert the resulting .cle image into a np.array to count objects within each class
    image_array = np.array(result, dtype=np.int8)

    # Create masks for each class
    background_mask = image_array == 0
    out_of_focus_mask = image_array == 1
    in_focus_mask = image_array == 2

    # Label connected components in each mask
    background_labels = measure.label(background_mask, connectivity=2)
    out_of_focus_labels = measure.label(out_of_focus_mask, connectivity=2)
    in_focus_labels = measure.label(in_focus_mask, connectivity=2)
    
    # Store in-focus labels for merging
    all_in_focus_labels.append(in_focus_labels)



In [ ]:
all_in_focus_labels[0].dtype

In [ ]:
viewer = napari.Viewer(ndisplay=2)
for in_focus_label in all_in_focus_labels:
    viewer.add_labels(in_focus_label, name="In-focus organoids")

In [ ]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_labels(all_in_focus_labels[6], name="In-focus organoids")

In [ ]:
# TODO: Merge all labels from each well into a big image, so I can use dimensionality reduction and cluster plotter on one image.
# TODO: Load in focus labels and greyscale images
# TODO: Extract features, perform UMAP and clustering. See Napari Cluster plotter
# TODO: Assign conditions, treatments to each well to display as hue in a pairplot https://umap-learn.readthedocs.io/en/latest/basic_usage.html

In [8]:
# Define the directory containing your files
directory_path ="../output/Andrew/P1.2023-10-02-08-13-59/in_out_focus_masks"

# The following function will read all the images contained within the directory_path above
# and store them grouped by well_id.
images_per_well = read_images(directory_path)


In [ ]:
# This is just a check, it prints the grouped z-stacks
for well_id, files in images_per_well.items():
    print(f'Well {well_id}:')
    for file_path in files:
        print(file_path)

In [10]:
imgs_to_process = images_per_well["A01"]

for input_img in imgs_to_process:

    # Load one RGB image and transform it into grayscale (if needed) for APOC
    in_focus_label = tifffile.imread(input_img, is_ome=False)

viewer = napari.Viewer(ndisplay=2)
viewer.add_labels(in_focus_label, name="Focus classification")


<Labels layer 'Focus classification' at 0x2733ffe3a60>